In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime 
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly


mpl.rcParams['figure.figsize']=(16,9)
pd.set_option('display.max_rows', 500)
sns.set(style="darkgrid")

import plotly.graph_objects as go

In [ ]:
df_analyse=pd.read_csv('processed.csv', sep=';')

In [ ]:
df_analyse.sort_values('Date',ascending=True).tail()

In [ ]:
country_list = df_analyse.columns[1:]

In [ ]:
def quick_plot(x_in, df, y_scale = 'log', slider = False, xname = ' ', yname = ' ', figname = ' '):
    """ Quick basic plot for quick static evaluation of a time series
    
        you can push selective columns of your data frame by .iloc[:,[0,6,7,8]]
        
        Parameters:
        ----------
        x_in : array 
            array of date time object, or array of numbers
        df_input : pandas dataframe 
            the plotting matrix where each column is plotted
            the name of the column will be used for the legend
        scale: str
            y-axis scale as 'log' or 'linear'
        slider: bool
            True or False for x-axis slider
        xname : str
            Title of the X axis
        yname : str
            Title of the Y axis
        figname : str
            Title of the figure
        
        Returns:
        ----------
        Return type: Figure
    
    """    
    fig = go.Figure()
    for each in df.columns:
        fig.add_trace(go.Scatter(x=x_in, 
                                 y=df[each], 
                                 name= each,
                                 mode= 'markers+lines',
                                 line_width = 1,
                                 marker_size = 3)
                     )
    fig.update_layout(xaxis_title = xname,
                      yaxis_title = yname,
                      width = 1000, 
                      height = 800,
                      title={
                        'text': figname,
                        'y':0.9,
                        'x':0.5,
                        'xanchor': 'center',
                        'yanchor': 'top'}
                    )
    fig.update_yaxes(type = y_scale)
    fig.update_xaxes(tickangle=-45,
                 nticks=20,
                 tickfont=dict(size=14,color="#7f7f7f")
                )    
    if slider==True:
        fig.update_layout(xaxis_rangeslider_visible=True)
    fig.show()

In [ ]:
#to check the helper function
quick_plot(df_analyse.Date,
           df_analyse.iloc[:,1:],
           y_scale='log',
           slider=True,
           xname = 'Date',
           yname = 'Confirmed infected people (source: John Hopkins csse) (log scale)',
           figname = 'Corona confirmed cases'
          )

In [ ]:
# Compare confirmed cases from some 'threshold' value
threshold = 150 

In [ ]:
compare_list = []
for pos, each in enumerate(df_analyse.columns[1:]):
    compare_list.append(np.array(df_analyse[each][df_analyse[each]>threshold]))  

In [ ]:
pd_sync_timelines = pd.DataFrame(compare_list, index = df_analyse.columns[1:]).T

In [ ]:
pd_sync_timelines['day'] = np.arange(pd_sync_timelines.shape[0])

In [ ]:
pd_sync_timelines.head()

In [ ]:
quick_plot(pd_sync_timelines.day,
           pd_sync_timelines.iloc[:,:-1],
           y_scale='log',
           slider=True,
           xname = 'Time in days',
           yname = 'Confirmed infected people (source: John Hopkins csse) (log scale)',
           figname = 'Comparison of corona cases'
          )

# Doubling Rate

In [ ]:
def doubling_rate(N_0,t,T_d):
    return N_0*np.power(2,t/T_d)

In [ ]:
max_days=100

In [ ]:
norm_slopes={
    'doubling every day':doubling_rate(100,np.arange(max_days),1),
    'doubling every two days':doubling_rate(100,np.arange(max_days),2),
    'doubling every 4 days':doubling_rate(100,np.arange(max_days),4),
    'doubling every 10 days':doubling_rate(100,np.arange(max_days),10),
}

In [ ]:
pd_sync_timelines_w_slope = pd.concat([pd.DataFrame(norm_slopes),pd_sync_timelines], axis=1)
pd_sync_timelines_w_slope.head()

In [ ]:
quick_plot(pd_sync_timelines_w_slope.day,
           pd_sync_timelines_w_slope.iloc[:,1:9],
           y_scale='log',
           slider=True,
           xname = 'Time in days',
           yname = 'Confirmed infected people (source: John Hopkins csse) (log scale)',
           figname = 'Corona infection rate of different countries'           
          )

In [ ]:
pd_sync_timelines_w_slope.to_csv('COVID_small_sync_timeline_table.csv',sep=';',index=False)

In [ ]:
from sklearn import linear_model
reg  = linear_model.LinearRegression(fit_intercept=False)

In [ ]:
l_vec = len(df_analyse['Italy'])
X=np.arange(l_vec-10).reshape(-1, 1) # Feature(training) 
y=np.log(np.array(df_analyse['Italy'][10:])) #Targets(training)

In [ ]:
reg.fit(X,y) # Train the model

In [ ]:
X_hat = np.arange(l_vec).reshape(-1, 1) #Feature(test)
y_hat = reg.predict(X_hat) # Prediction

In [ ]:
LR_inspect = df_analyse[['Italy','Date']].copy()
LR_inspect['prediction'] = np.exp(y_hat)
LR_inspect.head()

In [ ]:
fig = go.Figure()
quick_plot(LR_inspect.Date,
           LR_inspect.iloc[:,::2],
           y_scale='log',
           slider=True,
           xname = 'Date',
           yname = 'Infected people (source: John Hopkins csse) (log scale)',
           figname = 'Linear regression'          
          )

# Doubling Rate - Piecewise Linear Regression

In [ ]:
from sklearn import linear_model
from scipy import signal

In [ ]:
reg  = linear_model.LinearRegression(fit_intercept=True)

In [ ]:
l_vec = len(df_analyse['Italy'])
X=np.arange(l_vec-10).reshape(-1, 1)
y=np.array(df_analyse['Italy'][10:])

In [ ]:
df_analyse=pd.read_csv('processed.csv',sep=';',parse_dates=[0])  
country_list=df_analyse.columns[1:]

In [ ]:
filter_cols = []
for each in country_list:
    df_analyse[each+'_filter']=signal.savgol_filter(df_analyse[each],
                           5, # Window size used for filtering
                           1) # Order of fitted polynomial
    filter_cols.append(each+'_filter')

In [ ]:
start_pos=5
quick_plot(df_analyse.Date[start_pos:],
           df_analyse[filter_cols].iloc[start_pos:,:], #['US','US_filter']
           y_scale='log',
           slider=True,
           xname = 'Date',
           yname = 'Number of cases (filtered) (log scale)',
           figname = 'Filtered number of cases')

In [ ]:
df_analyse.head()

In [ ]:
def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate'''
    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)
    
    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_
    
    return intercept/slope

In [ ]:
def doubling_time(in_array):
    ''' Use a classical doubling time formula, see https://en.wikipedia.org/wiki/Doubling_time '''
    y = np.array(in_array)
    return len(y)*np.log(2)/np.log(y[-1]/y[0])

In [ ]:
# Calculate slope of regression of last x days
# Use always a limited number of days to approximate the triangle
days_back = 3  # This gives a smoothing effect
for pos,country in enumerate(country_list):
    df_analyse[country+'_DR']=df_analyse[country].rolling(
                                window=days_back,
                                min_periods=days_back).apply(get_doubling_time_via_regression, raw=False)

In [ ]:
# Run on all filtered data
days_back = 3 # this gives a smoothing effect
for pos,country in enumerate(filter_cols):
    df_analyse[country+'_DR']=df_analyse[country].rolling(
                                window=days_back,
                                min_periods=days_back).apply(get_doubling_time_via_regression, raw=False)

In [ ]:
# Cross check the math 
df_analyse['Italy_DR_math']=df_analyse['Italy'].rolling(
                                window=days_back,
                                min_periods=days_back).apply(doubling_time, raw=False)
df_analyse.columns

In [ ]:
start_pos=40
quick_plot(df_analyse.Date[start_pos: start_pos+30],
           df_analyse.iloc[start_pos: start_pos+30,[11,12,13,14,15]],
           y_scale='linear',
           slider=True,
           xname = 'Date',
           yname = 'Doubling rate',
           figname = 'Doubling rate'
          )

In [ ]:
start_pos=40
quick_plot(df_analyse.Date[start_pos: start_pos+30],
           df_analyse.iloc[start_pos:start_pos+30, [16,17,18,19,20]],
           y_scale='linear',
           slider=True,
           xname = 'Date',
           yname = 'Doubling rate(filtered data)',
           figname = 'Doubling rate for filtered data'
          )